#### Aug. 9, 2018 1pm

In [ ]:
import numpy   as np
import pandas  as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing   import scale, StandardScaler
from sklearn.decomposition   import PCA, TruncatedSVD

import pickle
from nltk.corpus import stopwords

import re

---

### Large Pickled Dataset



In [ ]:
%%time
df = pd.read_pickle("newRev_VegCols_US.pkl")

In [ ]:
df['vegFriendly'].value_counts()

In [ ]:
two = df['vegFriendly'] == 2

In [ ]:
%%time
df[two] = 1

In [ ]:
X_df = df.drop(['vegFriendly', 'Vegan', 'Vegetarian'], axis=1)
y_df = df['vegFriendly']

In [ ]:
X_df.info()

In [ ]:
X_df.iloc[:, 5] = X_df['text'].apply( lambda rev: re.sub(r'(\d+)', '', rev) )

In [ ]:
X_df.iloc[:, 5] = X_df['text'].apply( lambda rev: re.sub(r'^[a]*[a-zA-Z]*$', '', rev) )

In [ ]:
X_df.iloc[:, 5] = X_df['text'].apply( lambda rev: rev.replace('_', '') )

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3)

In [ ]:
print( X_train.shape, X_test.shape)

---

### Manipulate data sets

In [ ]:
stopWords = list(stopwords.words('english'))
stopWords.extend(['good', 'excellent', 'best', 'like', 'place', 'really', 'ordered', 'amazing', 'fantastic'])

vectorizer = TfidfVectorizer(stop_words=stopWords)

#### Vectorize Review Text

In [ ]:
%%time
X_train_term  = vectorizer.fit_transform(X_train['text'])

In [ ]:
X_train_term.shape

In [ ]:
ftrs = vectorizer.get_feature_names()

In [ ]:
ftrs

#### Reduce Dimensionality

In [ ]:
svd = TruncatedSVD(n_components=30, n_iter=7)

In [ ]:
%%time
X_train_term_svd = svd.fit_transform(X_train_term)  

In [ ]:
svd.singular_values_

In [ ]:
s = svd.singular_values_

x = [i+1 for i in range(len(s))]
plt.plot(x, s, marker='o')
plt.xlabel("Singular Values")
plt.ylabel("Value");

##### Percentage of Variance Explained

In [ ]:
svd.explained_variance_ratio_.sum()*100

---

#### Random Forest - Train

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RF_model = RandomForestClassifier(n_jobs=-1)

In [ ]:
%%time
RF_model.fit(X_train_term_svd, y_train)

---

#### Random Forest - Test

In [ ]:
%%time
X_test_term = vectorizer.transform(X_test['text'])

In [ ]:
X_test_term.shape

In [ ]:
%%time
X_test_term_svd = svd.transform(X_test_term)

In [ ]:
X_test_term_svd.shape

In [ ]:
%%time
pred = RF_model.predict(X_test_term_svd)

In [ ]:
len(pred)

In [ ]:
pred[:30]

In [ ]:
np.unique(pred)

In [ ]:
import itertools 
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
    """
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#     print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
lbls = ['Chinese', 'Indian', 'Mexican', 'Vegan', 'Vegetarian']

cm = confusion_matrix(y_test, pred, labels=lbls)

plot_confusion_matrix(cm,lbls)

In [ ]:
cm

In [ ]:
conf_mtx = dict()
metrics  = dict()
for i in range(cm.shape[0]):
    tp = cm[i,i]
    fn = cm[i,:].sum() - tp
    fp = cm[:,i].sum() - tp
    tn = cm.sum() - tp - fn - fp
    conf_mtx[ lbls[i] ] = np.array([[tn, fp], [fn, tp]])
    
    acc = (tp + tn) / conf_mtx[lbls[i]].sum()
    prc = tp / (tp + fp)
    rec = tp / (tp + fn)
    f1s = 2*tp / (2*tp + fp + fn)
    metrics[lbls[i]] = [acc, prc, rec, f1s]

In [ ]:
conf_mtx[lbls[0]]

In [ ]:
conf_mtx

In [ ]:
metrics

# === === === === === === === === === === === === === === === 

In [ ]:
classWeight = [{0: 1, 1: 1}, {0: 1, 1: 2}, {0: 1, 1: 1}, {0: 1, 1: 4}, {0: 1, 1: 4}]

In [ ]:
classWeight = {'Chinese': {0: 1, 1: 1}, 'Indian': {0: 1, 1: 2}, 'Mexican': {0: 1, 1: 1}, 'Vegan': {0: 1, 1: 4}, 'Vegetarian': {0: 1, 1: 4}}
classWeight

In [ ]:
classWeight = {'Chinese': 1, 'Indian': 2, 'Mexican': 1, 'Vegan': 4, 'Vegetarian': 4}
classWeight

In [ ]:
RF_model = RandomForestClassifier(n_jobs=-1, class_weight = classWeight)

In [ ]:
RF_model.fit(X_train_term_svd, y_train)

In [ ]:
pred = RF_model.predict(X_test_term_svd)

In [ ]:
lbls = ['Chinese', 'Indian', 'Mexican', 'Vegan', 'Vegetarian']

cm = confusion_matrix(y_test, pred, labels=lbls)

plot_confusion_matrix(cm,lbls)

#### Test Set: Class Balance

In [ ]:
y_test.value_counts()

In [ ]:
x = y_test.value_counts().values
lbls = list(y_test.value_counts().index)
plt.bar(range(len(x)), height=x, tick_label = lbls);

#### Train Set: Class Balance

In [ ]:
y_train.value_counts()

In [ ]:
x = y_train.value_counts().values
lbls = list(y_train.value_counts().index)
plt.bar(range(len(x)), height=x, tick_label = lbls);

---

### Train data PREDICTION

In [ ]:
pred2 = m_kmeans.predict(X_train_term_svd)

In [ ]:
catPred(m_kmeans, y_train, pred2)

In [ ]:
top_pred_words(m_kmeans, X_train_term, ftrs, pred2)

---

### Top Words for Cluster Centroids

In [ ]:
center_top_words(m_kmeans, m_svd, ftrs, 15)

In [ ]:
# https://github.com/gSchool/dsi-solns-g69/blob/master/clustering/pair_part1_kmeans.py

In [ ]:
pred

In [ ]:
# Print out the reviews of a random sample of the restaurants assigned to each
# cluster to get a sense of the category.

print("\nRandom sample of reviews in each cluster")
assigned_cluster = m_kmeans.transform(X_train_term_svd)

In [ ]:
assigned_cluster[:5,:]

In [ ]:
assigned_cluster = assigned_cluster[:10,:].argmin(axis=1)
assigned_cluster

In [ ]:
pred2[:10]

In [ ]:
m_kmeans.cluster_centers_

In [ ]:
vectorizer.get_feature_names()

In [ ]:
!pip install -U sklearn

In [ ]:
import sklearn

In [ ]:
sklearn.__version__

In [ ]:
! pip install -U scikit-learn